In [1]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

c:\Users\satsathi\FSDS_FT\localrepo\roi\using_detectron2\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch:  1.13 ; cuda:  cpu


In [2]:
import cv2

In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg = get_cfg()
cfg.MODEL.DEVICE='cpu'
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

In [4]:
img = cv2.imread('../mask_preparation/pic/clear.JPG')
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
# using masking
mask = cv2.imread('../mask_preparation/json/clear_json/label.png')
mask[:, :640, :] = 0 # Mask all left half region
cv2.imshow("Mask", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

img2gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)                 #Convert mask to grayscale
ret,mask = cv2.threshold(img2gray,0,255,cv2.THRESH_BINARY)         #Parts greater than 0 become white and parts less than or equal to 0 become black
masked_img = cv2.bitwise_and(img,img, mask=mask)                #Press bit and 0 to directly turn black, and the white part pasted on the original drawing remains unchanged

cv2.imshow("After applying mask", masked_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
# Use masked image for inferencing
outputs = predictor(masked_img)

v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("Inferencing Output",out.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

c:\Users\satsathi\FSDS_FT\localrepo\roi\using_detectron2\env\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
from PIL import Image
from detectron2.utils.visualizer import ColorMode
# gen_path = pathlib.Path.cwd()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu" 
  
cam = cv2.VideoCapture('KBsqQez-O4w.mp4')
  
while(True): 
    ret, frame = cam.read()
    frame = frame[:, :, [2,1,0]]
    frame = Image.fromarray(frame) 
    frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)
    mask = cv2.imread('../mask_preparation/json/clear_json/label.png')
    mask[:, :640, :] = 0 # Mask all left half region
    img2gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)                 #Convert mask to grayscale
    ret,mask = cv2.threshold(img2gray,0,255,cv2.THRESH_BINARY)         #Parts greater than 0 become white and parts less than or equal to 0 become black
    masked_frame = cv2.bitwise_and(frame,frame, mask=mask)

    # result = model(frame,size=640)
    outputs = predictor(masked_frame)
    

    v = Visualizer(frame[:, :, ::-1],
                   metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), 
                   scale=0.5, 
                #    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow('Detectron2',out.get_image()[:, :, ::-1])

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
cam.release()
cv2.destroyAllWindows()
